In [12]:
import cv2
import mediapipe as mp
import numpy as np
import os

In [13]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [14]:
# 비디오 파일 이름
input_file = "D:\\Pitch\\sample_videos\\deGrom_home.mp4" # 여기에 비디오 경로 넣으면 됩니다!

# 비디오 파일의 경우 이것을 사용하세요:
cap = cv2.VideoCapture(input_file)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

In [15]:
# 출력 파일 경로 및 이름 설정
output_file = os.path.splitext(input_file)[0] + "_dotted.mp4"
out = cv2.VideoWriter(output_file,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

In [16]:
with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("비디오 파일을 처리하는데 실패했습니다.")
            break

        # 필요에 따라 성능 향상을 위해 이미지 작성을 불가능함으로 기본 설정합니다.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        # 포즈 주석을 이미지 위에 그립니다.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        
        # 팔꿈치 위치를 찍기
        if results.pose_landmarks:
            for landmark in results.pose_landmarks.landmark:
                if landmark.HasField('visibility'):
                    if landmark.visibility < 0.5:
                        continue
                if landmark.HasField('presence'):
                    if landmark.presence < 0.5:
                        continue
                # 왼쪽 팔꿈치
                if landmark.type == mp_pose.PoseLandmark.LEFT_ELBOW:
                    cv2.circle(image, (int(landmark.x * image_width), int(landmark.y * image_height)), 5, (0, 255, 0), -1)
                # 오른쪽 팔꿈치
                if landmark.type == mp_pose.PoseLandmark.RIGHT_ELBOW:
                    cv2.circle(image, (int(landmark.x * image_width), int(landmark.y * image_height)), 5, (0, 0, 255), -1)

        # 프레임을 출력 파일에 작성
        out.write(image)

        cv2.imshow('MediaPipe Pose', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()
out.release()  # 출력 파일 닫기
cv2.destroyAllWindows()

비디오 파일을 처리하는데 실패했습니다.


In [6]:
import cv2

cap = cv2.VideoCapture(output_file)

if (cap.isOpened()== False): 
    print("비디오를 열 수 없습니다.")
  
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        cv2.imshow('Frame', frame)
        # q 키를 눌러 비디오 재생을 중지합니다.
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else: 
        break

cap.release()
cv2.destroyAllWindows()

# q 누르면 비디오 중지됩니다!!! 제발!!!